<a href="https://colab.research.google.com/github/scaairesearch/da_cv_fer/blob/main/Testing_Cleaned_Synthetic_Data_2503.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/scaairesearch/da_cv_fer.git
import os
os.getcwd()
os.chdir('da_cv_fer')

Cloning into 'da_cv_fer'...
remote: Enumerating objects: 373, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 373 (delta 80), reused 67 (delta 32), pack-reused 209
Receiving objects: 100% (373/373), 26.53 MiB | 16.67 MiB/s, done.
Resolving deltas: 100% (194/194), done.


In [2]:
!pip install -q -r requirements.txt # -q is for quiet install
# for some reason the below does not get installed with requirements
!pip install -q opendatasets
!pip install -q facenet_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.7/225.7 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [6]:
#@title loading libraries
from da_cv_fer.data_config import DataConfig
from da_cv_fer.ds_sfew import DatasetSFEW
from da_cv_fer.ds_sfew_crop import DatasetSFEWCROP
from da_cv_fer.ds_expw_crop import EXPWCROP
from da_cv_fer.ds_expw import EXPW
from da_cv_fer.utils import *
from da_cv_fer.model_DANN import DANN
from da_cv_fer.model_DANN_Bespoke import DANNBespoke
from da_cv_fer.run_config import RunConfig
from da_cv_fer.train import *
import os
import zipfile

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from tqdm import tqdm # for beautiful model training updates

import matplotlib.pyplot as plt # for plots

import time
from pathlib import Path

from copy import deepcopy
import numpy as np


# importing libraries
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import collections
import cv2
import os

In [3]:
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cpu


In [8]:
#@title Loading models to evaluate
# non_dann_model_sfew_name =  'non_dann_sfew_expw_14_03.pt'
dataconfig_inference = DataConfig()

In [9]:
# non_dann_model_inference = DANNBespoke(num_classes=7, backbone='mobilenet', attention=False, dropout=0)
# model_dir = Path('/content/drive/MyDrive/SCAAI_FER/Models/non_dann_sfew_expw_14_03.pt')
model_dates = ['03_03','14_03']
inference_models ={}
for model_date in model_dates:
  model_name = f'model_{model_date}'
  model_path = f'/content/drive/MyDrive/CV_FER/models/non_dann_sfew_expw/non_dann_sfew_expw_{model_date}.pt'
  try:
    inference_models[model_name] = DANNBespoke(num_classes=7, backbone='mobilenet', attention=False, dropout=0)
    inference_models[model_name].load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    print(f'{model_name} loaded successfully')
  except Exception as e:
    print(f'Problem in loading {model_name}: {e}')


# model_dir = '/content/drive/MyDrive/CV_FER/models/non_dann_sfew_expw/non_dann_sfew_expw_14_03.pt'



# try:
#     non_dann_model_inference.load_state_dict(torch.load(model_dir, map_location=torch.device(device)))
#     print(f'{model_dir} loaded successfully')
# except Exception as e:
#     print(f'Problem in loading {model_dir}: {e}')


Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 62.2MB/s]


feature extractor backbone created using mobilenet model
model_03_03 loaded successfully
feature extractor backbone created using mobilenet model
model_14_03 loaded successfully


In [10]:
inference_models.keys()

dict_keys(['model_03_03', 'model_14_03'])

In [11]:
# Define a mapping from numerical index to emotion label
labels_map = {
    "0": "Angry",
    "1": "Disgust",
    "2": "Fear",
    "3": "Happy",
    "4": "Sad",
    "5": "Surprise",
    "6": "Neutral"
}

In [12]:
import os

def get_files_in_folder(folder_path):
    files = []
    for file in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file)):
            files.append(file)
    return files


In [13]:
#@title Cropped Images
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
import IPython.display as display
from facenet_pytorch import MTCNN


synthetic_image_base_path = '/content/drive/MyDrive/CV_FER/dataset'
# dict_synthetic_image_folder = {'1':("synthetic_test_data_tranche_1","synthetic_test_data_tranche_1_cropped"),
#                                "2":("synthetic_test_data_tranche_2","synthetic_test_data_tranche_2_cropped")}

dict_synthetic_image_folder = {'1':("converged_synthethic_data_cleaned","converged_synthethic_data_cleaned_cropped")}

# initiatate MTCNN
mtcnn = MTCNN(image_size=224)

df_columns = ['image_name','cropped_image','tranche','emotion_gt','emotion_03_03','emotion_14_03']
image_df = pd.DataFrame(columns=df_columns)

for tranche, (orig_folder,cropped_folder) in dict_synthetic_image_folder.items():
  orig_path = os.path.join(synthetic_image_base_path, orig_folder)
  files_list = get_files_in_folder(orig_path)
  # print(files_list)
  for filename in files_list:
    image_path = os.path.join(orig_path, filename)
    img = Image.open(image_path).convert('RGB')

    #cropped
    img_cropped = mtcnn(img,
                        save_path=os.path.join(synthetic_image_base_path, cropped_folder, filename))
    # Scale the values from [-1, 1] to [0, 1]
    normalized_image_cropped = (img_cropped + 1) / 2
    normalized_image_cropped = torch.clamp(normalized_image_cropped, 0, 1)

    emotion_gt_label = filename.split('_')[0]

    column_values ={
        'image_name':filename,
        'cropped_image':normalized_image_cropped,
        'tranche':tranche,
        'emotion_gt':emotion_gt_label,
        'emotion_03_03':9,
        'emotion_14_03':9
    }
    image_df = image_df.append(column_values, ignore_index=True)

image_df.head()

,image_name,cropped_image,tranche,emotion_gt,emotion_03_03,emotion_14_03
0,Angry_Woman_Old.png,"[[[tensor(0.6543), tensor(0.6465), tensor(0.63...",1,Angry,9,9
1,Angry_Man_Old.png,"[[[tensor(0.9902), tensor(0.9902), tensor(0.98...",1,Angry,9,9
2,Happy_Male_Light.png,"[[[tensor(0.2871), tensor(0.2871), tensor(0.28...",1,Happy,9,9
3,Happy_Woman_Old.png,"[[[tensor(0.6074), tensor(0.6270), tensor(0.65...",1,Happy,9,9
4,Neutral_Woman_Jewels.png,"[[[tensor(0.6230), tensor(0.5449), tensor(0.49...",1,Neutral,9,9


In [14]:
len(image_df)

89

In [15]:
## inferencing


for inference_model_name, inference_model in inference_models.items():
    print(f"Inference Model: {inference_model_name}")
    model_date = ('_').join(inference_model_name.split("_")[-2:])
    print(f"Model Date: {model_date}")
    column_name = f'emotion_{model_date}'
    print(f"Column Name: {column_name}")

    inference_model.eval()  # Set the model to evaluation mode

    for index, row in image_df.iterrows():
        image = row['cropped_image']
        image = image.to(device)
        with torch.no_grad():
          image = image.unsqueeze(0) # batch  = 1
          output,_,_ = inference_model(image)
          _, predicted_index = torch.max(output, 1)
          predicted_label = labels_map[str(predicted_index.item())]  # Convert numerical index to emotion label
          print(f"Predicted Emotion: {predicted_label}")
          print("Ground Truth:",row['emotion_gt'])
          df_row = image_df.loc[index]
          df_row[column_name] = predicted_label
          image_df.loc[index] = df_row

print("Inferencing complete")

Inference Model: model_03_03
Model Date: 03_03
Column Name: emotion_03_03
Predicted Emotion: Disgust
Ground Truth: Angry
Predicted Emotion: Disgust
Ground Truth: Angry
Predicted Emotion: Happy
Ground Truth: Happy
Predicted Emotion: Happy
Ground Truth: Happy
Predicted Emotion: Fear
Ground Truth: Neutral
Predicted Emotion: Sad
Ground Truth: Neutral
Predicted Emotion: Fear
Ground Truth: Surprised
Predicted Emotion: Fear
Ground Truth: Surprised
Predicted Emotion: Fear
Ground Truth: Sad
Predicted Emotion: Fear
Ground Truth: Sad
Predicted Emotion: Fear
Ground Truth: Disgust
Predicted Emotion: Fear
Ground Truth: Disgust
Predicted Emotion: Happy
Ground Truth: Neutral
Predicted Emotion: Happy
Ground Truth: Neutral
Predicted Emotion: Happy
Ground Truth: Neutral
Predicted Emotion: Happy
Ground Truth: Neutral
Predicted Emotion: Fear
Ground Truth: Neutral
Predicted Emotion: Sad
Ground Truth: Neutral
Predicted Emotion: Disgust
Ground Truth: Neutral
Predicted Emotion: Disgust
Ground Truth: Neutral
Pr

In [16]:
image_df

,image_name,cropped_image,tranche,emotion_gt,emotion_03_03,emotion_14_03
0,Angry_Woman_Old.png,"[[[tensor(0.6543), tensor(0.6465), tensor(0.63...",1,Angry,Disgust,Angry
1,Angry_Man_Old.png,"[[[tensor(0.9902), tensor(0.9902), tensor(0.98...",1,Angry,Disgust,Disgust
2,Happy_Male_Light.png,"[[[tensor(0.2871), tensor(0.2871), tensor(0.28...",1,Happy,Happy,Happy
3,Happy_Woman_Old.png,"[[[tensor(0.6074), tensor(0.6270), tensor(0.65...",1,Happy,Happy,Happy
4,Neutral_Woman_Jewels.png,"[[[tensor(0.6230), tensor(0.5449), tensor(0.49...",1,Neutral,Fear,Surprise
...,...,...,...,...,...,...
84,Disgust_Male_Beard_White.png,"[[[tensor(0.2754), tensor(0.2871), tensor(0.27...",1,Disgust,Angry,Angry
85,Disgust_Male_Beard_European.png,"[[[tensor(0.3965), tensor(0.4004), tensor(0.41...",1,Disgust,Angry,Disgust
86,Disgust_Female_Makeup_Asian.png,"[[[tensor(0.6309), tensor(0.4941), tensor(0.44...",1,Disgust,Happy,Surprise
87,Disgust_Female_Cap_African.png,"[[[tensor(0.7559), tensor(0.7637), tensor(0.77...",1,Disgust,Fear,Disgust


In [17]:
results_df = image_df.drop(columns=['cropped_image'])
results_df.head()

,image_name,tranche,emotion_gt,emotion_03_03,emotion_14_03
0,Angry_Woman_Old.png,1,Angry,Disgust,Angry
1,Angry_Man_Old.png,1,Angry,Disgust,Disgust
2,Happy_Male_Light.png,1,Happy,Happy,Happy
3,Happy_Woman_Old.png,1,Happy,Happy,Happy
4,Neutral_Woman_Jewels.png,1,Neutral,Fear,Surprise


In [18]:
results_df.to_excel('results_selective.xlsx', index=False)

In [19]:
# Creating the Metrics
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, classification_report


def print_metrics(true_labels, predicted_labels):
  # Compute accuracy
  accuracy = accuracy_score(true_labels, predicted_labels)
  print("Accuracy:", accuracy)

  # Compute precision
  precision = precision_score(true_labels, predicted_labels, average='weighted')
  print("Precision:", precision)

  # Generate confusion matrix
  conf_matrix = confusion_matrix(true_labels, predicted_labels)
  print("Confusion Matrix:")
  print(conf_matrix)

  # Generate classification report
  class_report = classification_report(true_labels, predicted_labels)
  print("Classification Report:")
  print(class_report)


In [20]:
# predicted_labels_03_03_1 = results_df['emotion_03_03'][image_df['tranche'] == '1'].tolist()
# predicted_labels_14_03_1 =  results_df['emotion_14_03'][image_df['tranche'] == '1'].tolist()

# predicted_labels_03_03_2  = results_df['emotion_03_03'][image_df['tranche'] == '2'].tolist()
# predicted_labels_14_03_2  =  results_df['emotion_14_03'][image_df['tranche'] == '2'].tolist()

# true_labels_1 = results_df['emotion_gt'][image_df['tranche'] == '1'].tolist()
# true_labels_2 = results_df['emotion_gt'][image_df['tranche'] == '2'].tolist()

predicted_labels_03_03 = results_df['emotion_03_03'].tolist()
predicted_labels_14_03 = results_df['emotion_14_03'].tolist()

true_labels = results_df['emotion_gt'].tolist()

In [21]:
# # Tranche 1 : 03_03 v GT
# print_metrics(true_labels_1, predicted_labels_03_03_1)

In [22]:
# # Tranche 1 : 14_03 v GT
# print_metrics(true_labels_1, predicted_labels_14_03_1)

In [23]:
# # Tranche 2 : 03_03 v GT
# print_metrics(true_labels_2, predicted_labels_03_03_2)

In [24]:
# # Tranche 2 : 14_03 v GT
# print_metrics(true_labels_2, predicted_labels_14_03_2)

In [25]:
# All : 03_03 v GT
print_metrics(true_labels, predicted_labels_03_03)

Accuracy: 0.21348314606741572
Precision: 0.173592761795009
Confusion Matrix:
[[ 2  5  4  0  1  1  1  0]
 [ 3  1  5  3  0  2  1  0]
 [ 0  0  1  1  0  3  3  0]
 [ 0  0  0 12  0  1  1  0]
 [ 0  2  2  4  0  3  1  0]
 [ 2  4  5  0  0  0  3  0]
 [ 2  0  2  0  0  3  3  0]
 [ 0  0  2  0  0  0  0  0]]
Classification Report:
              precision    recall  f1-score   support

       Angry       0.22      0.14      0.17        14
     Disgust       0.08      0.07      0.07        15
        Fear       0.05      0.12      0.07         8
       Happy       0.60      0.86      0.71        14
     Neutral       0.00      0.00      0.00        12
         Sad       0.00      0.00      0.00        14
    Surprise       0.23      0.30      0.26        10
   Surprised       0.00      0.00      0.00         2

    accuracy                           0.21        89
   macro avg       0.15      0.19      0.16        89
weighted avg       0.17      0.21      0.19        89



In [26]:
# All : 14_03 v GT
print_metrics(true_labels, predicted_labels_14_03)

Accuracy: 0.38202247191011235
Precision: 0.34494251067284776
Confusion Matrix:
[[ 4  7  1  0  2  0  0  0]
 [ 1  9  0  0  4  0  1  0]
 [ 0  2  1  0  3  2  0  0]
 [ 0  1  0 12  0  1  0  0]
 [ 2  1  0  1  3  3  2  0]
 [ 3  5  2  0  0  0  4  0]
 [ 0  1  0  1  3  0  5  0]
 [ 1  0  0  0  1  0  0  0]]
Classification Report:
              precision    recall  f1-score   support

       Angry       0.36      0.29      0.32        14
     Disgust       0.35      0.60      0.44        15
        Fear       0.25      0.12      0.17         8
       Happy       0.86      0.86      0.86        14
     Neutral       0.19      0.25      0.21        12
         Sad       0.00      0.00      0.00        14
    Surprise       0.42      0.50      0.45        10
   Surprised       0.00      0.00      0.00         2

    accuracy                           0.38        89
   macro avg       0.30      0.33      0.31        89
weighted avg       0.34      0.38      0.35        89



Later part of the notebook not used